In [13]:
from google.colab import files
datafiles = files.upload()

Saving train_LZdllcl.csv to train_LZdllcl (1).csv


In [14]:
from google.colab import files
datafiles = files.upload()

Saving test_2umaH9m.csv to test_2umaH9m (1).csv


In [0]:
import pandas as pd
from fancyimpute import KNN, IterativeImputer
from imblearn.over_sampling import SMOTE

In [0]:
train = pd.read_csv('train_LZdllcl.csv')

In [176]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 14 columns):
employee_id             54808 non-null int64
department              54808 non-null object
region                  54808 non-null object
education               52399 non-null object
gender                  54808 non-null object
recruitment_channel     54808 non-null object
no_of_trainings         54808 non-null int64
age                     54808 non-null int64
previous_year_rating    50684 non-null float64
length_of_service       54808 non-null int64
KPIs_met >80%           54808 non-null int64
awards_won?             54808 non-null int64
avg_training_score      54808 non-null int64
is_promoted             54808 non-null int64
dtypes: float64(1), int64(8), object(5)
memory usage: 5.9+ MB


In [177]:
train.isnull().sum()

employee_id                0
department                 0
region                     0
education               2409
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    4124
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
is_promoted                0
dtype: int64

In [0]:
train['education'] = pd.factorize(train['education'])[0]
train['department'] = pd.factorize(train['department'])[0]
train['region'] = pd.factorize(train['region'])[0]
train['gender'] = pd.factorize(train['gender'])[0]
train['recruitment_channel'] = pd.factorize(train['recruitment_channel'])[0]

In [0]:
data_imputation = train.loc[:,['education','previous_year_rating']]

In [0]:
X_filled_ii = pd.DataFrame(IterativeImputer().fit_transform(data_imputation))

In [0]:
X_filled_ii.columns = ['education','previous_year_rating']

In [182]:
X_filled_ii.shape

(54808, 2)

In [0]:
train_new=train.drop(['education','previous_year_rating'], axis=1)

In [184]:
train_new.shape

(54808, 12)

In [0]:
concat_train=pd.concat([train_new,X_filled_ii],axis=1)

In [186]:
concat_train.shape

(54808, 14)

In [0]:
smt = SMOTE()

In [0]:
test = pd.read_csv('test_2umaH9m.csv')

In [189]:
concat_train.isnull().sum()

employee_id             0
department              0
region                  0
gender                  0
recruitment_channel     0
no_of_trainings         0
age                     0
length_of_service       0
KPIs_met >80%           0
awards_won?             0
avg_training_score      0
is_promoted             0
education               0
previous_year_rating    0
dtype: int64

In [0]:
train_copy = concat_train.copy()

In [0]:
X = train_copy.drop(['is_promoted'], axis=1)

In [192]:
X.shape

(54808, 13)

In [193]:
train_copy.columns

Index(['employee_id', 'department', 'region', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'length_of_service', 'KPIs_met >80%',
       'awards_won?', 'avg_training_score', 'is_promoted', 'education',
       'previous_year_rating'],
      dtype='object')

In [0]:
y = train_copy['is_promoted']

In [197]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54808 entries, 0 to 54807
Data columns (total 13 columns):
employee_id             54808 non-null int64
department              54808 non-null int64
region                  54808 non-null int64
gender                  54808 non-null int64
recruitment_channel     54808 non-null int64
no_of_trainings         54808 non-null int64
age                     54808 non-null int64
length_of_service       54808 non-null int64
KPIs_met >80%           54808 non-null int64
awards_won?             54808 non-null int64
avg_training_score      54808 non-null int64
education               54808 non-null float64
previous_year_rating    54808 non-null float64
dtypes: float64(2), int64(11)
memory usage: 5.4 MB


In [198]:
y.shape

(54808,)

In [0]:
X, y =smt.fit_sample(X, y)

(100280, 13)

In [201]:
y.shape

(100280,)

In [0]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size =0.2,random_state = 123)

In [0]:
tree = DecisionTreeClassifier()
rf = RandomForestClassifier()
bagg = BaggingClassifier(base_estimator=tree)
boost = AdaBoostClassifier(base_estimator=tree)

In [0]:
param_grid_tree = {'criterion':['gini','entropy'], 'max_depth':[3,5,7,9]}
param_grid_rf = {'n_estimators':[100,200,500], 'max_depth':[3,5,7,9]}
param_grid_bagg = {'n_estimators':[100,200,500]}
param_grid_boost = {'n_estimators':[100,200,500], 'learning_rate':[0.0001,0.0003,0.0005]}

In [0]:
best_model_tree = GridSearchCV(estimator=tree,param_grid=param_grid_tree, scoring='roc_auc', cv=5)

best_model_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, scoring='roc_auc',  cv=5)

best_model_bagg = GridSearchCV(estimator=bagg,param_grid=param_grid_bagg, scoring='roc_auc',  cv=5)

best_model_boost = GridSearchCV(estimator=boost, param_grid=param_grid_boost, scoring='roc_auc',cv=5)

In [144]:
best_model_tree.fit(X=X_train, y=y_train)

GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'criterion': ['gini', 'entropy'], 'max_depth': [3, 5, 7, 9]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='roc_auc', verbose=0)

In [145]:
best_model_tree.best_params_

{'criterion': 'entropy', 'max_depth': 9}

In [146]:
cross_val_score(best_model_tree.best_estimator_, X=X_train, y= y_train,cv=5)

array([0.91530072, 0.92527267, 0.91779371, 0.92633219, 0.91429818])

In [0]:
from sklearn.linear_model import LogisticRegression

In [0]:
classifier = LogisticRegression(random_state = 0)

In [206]:
classifier.fit(X_train,y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=0, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_pred_Lr= classifier.predict(X_test)

In [0]:
from xgboost import XGBClassifier
import numpy as np

In [240]:
X_test = X_test.as_matrix()
X_train = X_train.as_matrix()

AttributeError: ignored

In [0]:
XG_model = XGBClassifier()

In [0]:
col=concat_train.columns

In [210]:
XG_model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [0]:
y_pred_Xg= XG_model.predict(X_test)

In [212]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred_Lr))

              precision    recall  f1-score   support

           0       0.77      0.73      0.75     10195
           1       0.73      0.77      0.75      9861

   micro avg       0.75      0.75      0.75     20056
   macro avg       0.75      0.75      0.75     20056
weighted avg       0.75      0.75      0.75     20056



In [213]:
print(classification_report(y_test,y_pred_Xg ))

              precision    recall  f1-score   support

           0       0.93      1.00      0.96     10195
           1       1.00      0.92      0.96      9861

   micro avg       0.96      0.96      0.96     20056
   macro avg       0.96      0.96      0.96     20056
weighted avg       0.96      0.96      0.96     20056



In [0]:
test['education'] = pd.factorize(test['education'])[0]
test['department'] = pd.factorize(test['department'])[0]
test['region'] = pd.factorize(test['region'])[0]
test['gender'] = pd.factorize(test['gender'])[0]
test['recruitment_channel'] = pd.factorize(test['recruitment_channel'])[0]

In [215]:
test.isnull().sum()

employee_id                0
department                 0
region                     0
education                  0
gender                     0
recruitment_channel        0
no_of_trainings            0
age                        0
previous_year_rating    1812
length_of_service          0
KPIs_met >80%              0
awards_won?                0
avg_training_score         0
dtype: int64

In [0]:
test_imp = test.loc[:,['education','previous_year_rating']]

In [0]:
test_filled = pd.DataFrame(IterativeImputer().fit_transform(test_imp))

In [0]:
test_filled.columns = ['education','previous_year_rating']

In [219]:
test.shape

(23490, 13)

In [0]:
test_new = test.drop(['education','previous_year_rating'], axis=1)

In [0]:
concat_test=pd.concat([test_new,test_filled],axis=1)

In [244]:

concat_test.columns
concat_train.columns

Index(['employee_id', 'department', 'region', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'length_of_service', 'KPIs_met >80%',
       'awards_won?', 'avg_training_score', 'is_promoted', 'education',
       'previous_year_rating'],
      dtype='object')

In [230]:
concat_train.columns

Index(['employee_id', 'department', 'region', 'gender', 'recruitment_channel',
       'no_of_trainings', 'age', 'length_of_service', 'KPIs_met >80%',
       'awards_won?', 'avg_training_score', 'is_promoted', 'education',
       'previous_year_rating'],
      dtype='object')

In [0]:
is_promoted = XG_model.predict(concat_test.values)

In [0]:
submission = pd.DataFrame.from_dict(data={'Id': concat_test['employee_id'], 'is_promoted': is_promoted})

In [249]:
submission.head()

,Id,is_promoted
0,8724,1
1,74430,0
2,72255,0
3,38562,0
4,64486,0


In [0]:
submission.to_csv('submission.csv')
files.download('submission.csv')